## Q4
POS Tagging - HMM (50 points)
The training dataset is a subset of the Brown corpus, where each file contains sentences of
tokenized words followed by POS tags, and where each line contains one sentence:
https://www.dropbox.com/sh/havbkrjqzu9kpv6/AABVY0xRUvu-AO2TyftQUwCEa?dl=0
The test dataset (which is another subset of the Brown corpus, containing tokenized words
but no tags) is the following:


https://www.dropbox.com/s/5j62js3pa0z6z9e/science_sample.txt?dl=0
Information regarding the categories of the dataset can be found at:
https://www.dropbox.com/s/ujnz04e62e9603j/CONTENTS.txt?dl=0
Your task is to implement a part-of-speech tagger using a bigram HMM. Given an ob-
servation sequence of n words wn
1 , choose the most probable sequence of POS tags tn1
.

[Note: During training, for a word to be counted as unknown, the frequency of word in
training set should not exceed a threshold (e.g. 5). You can pick a threshold based on your
algorithm design.]


To obtain the tag unigram counts and the tag bigram counts, you will need to separate
out each tag from its word. For each sentence found in the training data, add a start token
and an end token to the beginning and the end of the sentence.


In [1]:
import os
import time
import re
import pickle
import random
import numpy as np
from collections import Counter

In [6]:
## get dataset and append start and end tag
def data_extractor(dataset):
    complete_data = []
    filename = os.getcwd() + "\\" + dataset 
    for text in os.listdir(filename):
        filename = filename + "\\" + text
        if ".DS_Store" not in filename:
            with open(filename, encoding = "utf8", errors = "ignore") as file:
                text_in_file = file.read()
                text_in_file = re.sub("\n+", "\n", text_in_file)
                text_in_file = re.sub("\t", "", text_in_file)
                text_in_file = text_in_file.split("./.")
                list_words = []
                for i in text_in_file:
                    if i not in ["\n", "", " "]:
                        i = i.strip("\n")
                        list_of_words = "<START> " + i + " <END>"
                        word_list = re.findall("\w+/\w+|\<START\>|\<END\>", list_of_words)
                        list_words.extend(word_list)
            complete_data.extend(list_words)
        filename = os.getcwd() + "\\" + dataset 
    return complete_data


    

list


## 4.1 
Obtain frequency counts from the collection of all the training les (counted together). You
will need the following types of frequency counts: word-tag counts, tag unigram counts,
and tag bigram counts. Lets denote these by C (wi ; ti), C (ti), and C (ti􀀀1 ; ti) respectively.
Report these quantities.


In [10]:
## find and count all the unigrams
text_in_file = data_extractor(dataset = "brown-copy")
complete_data_list = [i.split("/")[0] for i in text_in_file if "/" in i]
data_unigram_count = Counter(complete_data_list)
data_count_less_than_5 = {i: data_unigram_count[i] for i in data_unigram_count if data_unigram_count[i] < 5}

In [85]:
## change the less frequent words to "UNK" token, using replace function instead of list manipulation since it is faster
data_text = " ".join(text_in_file)
for index, i in enumerate(data_count_less_than_5):
    word = " " + i + "/"
    data_text = data_text.replace(word , " UNK/")
    if index % 1000 == 0:
        print (index)

In [ ]:
## count of word and tag
data_text_list = data_text.split(" ")
counter_word_tag = Counter(data_text_list)

In [ ]:
## count of only tags
tag_list = [i if i in ["<START>", "<END>"] else i.split("/")[1] for i in data_text_list]
counter_tag = Counter(tag_list)

In [ ]:
# count of bigrams tag
bigram_tag_list = [" ".join(tag_list[index:index + 2]) for index in range(len(tag_list[:-1]))]
bigram_counter_tag = Counter(bigram_tag_list)


## 4.2 
A transition probability is the probability of a tag given its previous tag. 

In [16]:
## calculate the probability of transition
prob_transition = {}
lambda_val = 0.1
smoothed_prob_transition = {}
for i in bigram_counter_tag:
    previous_tag = i.split(" ")[0]
    prob_transition[i] = (bigram_counter_tag[i] + lambda_val)/(counter_tag[previous_tag] + (lambda_val * len(counter_tag)))
    smoothed_prob_transition[previous_tag] = (lambda_val)/((counter_tag[previous_tag] +lambda_val * len(counter_tag)))

## 4.3
An emission probability is the probability of a given word being associated with a given tag.


In [17]:
## emission probability is the probability of a given word being associated with a given tag
prob_emission = {}
lambda_val = 0.1
smoothed_prob_emission = {}
for i in counter_word_tag:
    if i not in  ["<START>", "<END>"]:
        tag_of_word = i.split("/")[1]
        prob_emission[i] = (counter_word_tag[i] + lambda_val)/(counter_tag[tag_of_word] + (lambda_val * len(counter_word_tag)))
        smoothed_prob_emission[tag_of_word] = (lambda_val)/((counter_tag[tag_of_word] +lambda_val * len(counter_word_tag)))

## 4.4

Generate 5 random sentences using HMM. Output each sentence (with the POS tags) and
its probability of being generated.

In [100]:
## generate five random sentences based on emission and transition probability starting and ending with <START> and <END> yag 
list_5_rand_sentences = []
while (len(list_5_rand_sentences) != 5):
    prev_tag = "<START>"
    tag_drawn = " "
    rand_sententence = []
    while True:
        next_set_of_tags = [i for i in bigram_counter_tag if i.split(" ")[0] == prev_tag and i != "<END> <START>"]
        prob_next_tag = [prob_transition[i] for i in next_set_of_tags]
        tag_drawn = random.choices(next_set_of_tags,prob_next_tag)
        prev_tag = tag_drawn[0].split(" ")[1]
        if prev_tag == "<END>":
            break
        probable_words = [i for i in counter_word_tag if i not in ["<END>","<START>"] and i.split("/")[1] == prev_tag and "UNK" not in i]
        word_probabilities = [prob_emission[i] for i in probable_words]
        word_drawn = random.choices(probable_words,word_probabilities)
        prob_of_this_word = prob_emission[word_drawn[0]] * prob_transition[tag_drawn[0]]
        word_drawn[0] = word_drawn[0] + " " + str(prob_of_this_word)
        rand_sententence.extend(word_drawn)
    list_5_rand_sentences.append(" ".join(rand_sententence))


In [106]:
## print the sentences
for index, i in enumerate(list_5_rand_sentences):
    print ("Sentence ID:", index+1, "\n\t", i, "\n")
    sentence = i.split(" ")
    sent_list = [j.split("/")[0] for j in sentence if "/" in j]
    sent_str = " ".join(sent_list)
    print ("**Printing the sentence properly**\n\t",sent_str, "\n")

Sentence ID: 1 
	 John/np 0.0007098881965105562 William/np 0.0007291966409498122 of/in 0.018772209734886754 the/at 0.22266728573686068 grounds/nns 6.915242320338724e-05 of/in 0.07908709712941428 all/abn 0.00434994068179207 be/be 0.001543369745140279 with/in 0.002340103095761513 both/abx 0.0004348187807154439 The/at 0.007202984273656557 spite/nn 0.00015251024398632337 and/cc 0.05789187326335965 it/pps 0.007095342160281965 was/bedz 0.12110602133238504 implying/vbg 3.072770558043864e-05 

**Printing the sentence properly**
	 John William of the grounds of all be with both The spite and it was implying 

Sentence ID: 2 
	 who/wps 0.0005389002024534835 get/vb 0.0021519696210883033 his/pp 0.01681628796631599 British/jj 0.0002648576310265283 to/to 0.016668149746310208 understand/vb 0.0030966175475128834 Hanover/np 3.354784438502727e-05 

**Printing the sentence properly**
	 who get his British to understand Hanover 

Sentence ID: 3 
	 shaking/vbg 1.1680090182855517e-05 to/in 0.014613290979083

In [23]:
## get the test dataset and format it
def collect_testdata(filename): 
    with open(filename) as file:
        file_content = file.read()
    file_content = file_content.split("<EOS>")
    file_dict = {}
    for index, content in enumerate(file_content):
        list_content = content.split("\n")
        list_content = [i for i in list_content if re.match("\w+", i)]
        file_dict[index] = list_content
    return file_dict

filename = r"C:\Users\mm199\NLP\HW1\science_sample.txt"
file_dict = collect_testdata(filename)

## 4.5 Implement Viterbi Algorithm
For each word, output the tag derived using the
Viterbi algorithm in the following format(where each line contains no more than one pair):
<sentence ID=1>
word, tag
word, tag
:::
word, tag
<EOS>
<sentence ID=2>
word, tag
word, tag
:::
word, tag
<EOS>

Submit your code, a README.txt file explaining how to run your code. Please also
include your output file with the tag predictions in the format specified above.



In [79]:
## viterbi algorithm by just considering previous state
def viterbi_algo(test_data, counter_tag, prob_transition, prob_emission, smoothed_prob_transition, smoothed_prob_emission):
    len_of_test_data = len(test_data)
    total_unique_tags = [i for i in counter_tag if i not in ["<START>", "<END>"]]
    len_of_total_tags = len(total_unique_tags)
    pi = np.ndarray((len_of_test_data+1, len_of_total_tags+1))
    bp =  np.ndarray((len_of_test_data, len_of_total_tags))
    pi[0][:] = 1.0
    for i in range(len_of_test_data):
        for index1, tags1 in enumerate(total_unique_tags):
            max_tag_prob = -1
            if i == 0:
                all_tags = ["<START>"]
            else:
                all_tags = total_unique_tags
            for index2, tags2 in enumerate(all_tags):
                transition = tags2 + " " + tags1
                emission = test_data[i] + "/" + tags1
                if transition not in prob_transition or prob_transition[transition] == 0:
                    prob_transition[transition] = smoothed_prob_transition[tags2]
                else:
                    pass
                if emission not in prob_emission or prob_emission[emission] == 0:
                    prob_emission[emission] = smoothed_prob_emission[tags1]
                else:
                    pass
                prob = pi[i][index2] * prob_transition[transition] * prob_emission[emission]
                if prob > max_tag_prob:
                    max_tag_prob = prob
                    max_index = index2
            if max_tag_prob == 0:
                print (index1, i, tags1, index2, tags2)
            pi[i+1][index1] = max_tag_prob
            bp[i][index1] = max_index
    ## get the back pointer starting point
    last_array = pi[(len_of_test_data)][1:]
    for i, tags in enumerate(total_unique_tags):
        transition = tags + " " + "<END>"
        if transition not in prob_transition or prob_transition[transition] == 0:
            prob_transition[transition] = smoothed_prob_transition[tags]
        else:
            pass
        last_array[i] = last_array[i] * prob_transition[transition]
    bp_start = np.argmax(last_array)
    ## traverse from the end with the help of back pointer
    index = len_of_test_data - 1
    word_with_tag_result_list = [(test_data[index], total_unique_tags[bp_start])] ## initiate the tuple which contains word and its tag
    pointer = bp_start
    while (index != -1):
        pointer = int (bp[index][pointer])
        if index != 0:
            tuple_to_be_added = (test_data[index-1], total_unique_tags[pointer])
            word_with_tag_result_list.append(tuple_to_be_added)
        index -= 1
    return word_with_tag_result_list

In [80]:
## save the result in a list of list as a tuple format
final_tag_result = []
for i in range(len(file_dict)):
    test_data = file_dict[i]
    if test_data == []:
        break
    word_with_tag_result_list = viterbi_algo(test_data, counter_tag, prob_transition, prob_emission, smoothed_prob_transition, smoothed_prob_emission)
    final_tag_result.append(word_with_tag_result_list)

In [84]:
## write the output on a file
with open("output.txt", "w") as file:
    for index, each_sentence in enumerate(final_tag_result):
        start = "<sentence ID=%s>\n"%(index+1)
        file.write(start)
        for word,tag in each_sentence[::-1]:
            word_and_its_tag = (word + ", " + tag + "\n")
            file.write(word_and_its_tag)
        end = "<EOS>\n\n"
        file.write(end)